# AC Analysis and Bode Plots

This notebook covers small-signal AC analysis for frequency response characterization.

## Contents
1. AC Analysis Basics
2. Frequency Sweep Options
3. RC Low-Pass Filter Analysis
4. Bode Plot Generation
5. Stability Margins
6. LC and RLC Filters
7. Control Loop Analysis

In [1]:
import pulsim
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = [12, 5]
plt.rcParams['font.size'] = 11

ModuleNotFoundError: No module named 'pulsim'

## 1. AC Analysis Basics

AC analysis computes the small-signal frequency response of a circuit:

1. **DC Operating Point**: Find the steady-state DC solution
2. **Linearization**: Linearize nonlinear elements at the operating point
3. **Frequency Sweep**: Solve complex admittance matrix at each frequency

$$\mathbf{Y}(j\omega) \cdot \mathbf{V} = \mathbf{I}$$

Where $\mathbf{Y}(j\omega) = \mathbf{G} + j\omega\mathbf{C}$

In [ ]:
# AC analysis workflow
print("AC Analysis Steps:")
print("="*50)
print("1. Create circuit with AC source")
print("2. Configure ACOptions (frequency range, sweep type)")
print("3. Run ac_analysis() or ACAnalyzer")
print("4. Extract magnitude and phase from ACResult")
print("5. Generate Bode plots")

## 2. Frequency Sweep Options

```python
ACOptions:
    sweep_type   # Linear, Decade, Octave, List
    fstart       # Start frequency (Hz)
    fstop        # Stop frequency (Hz)
    npoints      # Number of points (or points per decade/octave)
    frequency_list  # For List sweep type
```

### Sweep Types

| Type | Description | Best For |
|------|-------------|----------|
| **Linear** | Equal frequency spacing | Narrow band analysis |
| **Decade** | Logarithmic by decades | Wide band (typical) |
| **Octave** | Logarithmic by octaves | Audio applications |
| **List** | Specific frequencies | Custom points |

In [ ]:
# Demonstrate frequency sweep types
fig, axes = plt.subplots(2, 2, figsize=(14, 8))

# Linear sweep
f_linear = np.linspace(100, 10000, 50)
axes[0, 0].stem(f_linear, np.ones_like(f_linear), basefmt=' ')
axes[0, 0].set_xlabel('Frequency (Hz)')
axes[0, 0].set_title('Linear Sweep: 100Hz to 10kHz, 50 points')
axes[0, 0].set_xlim([0, 11000])

# Decade sweep (10 points per decade)
decades = np.arange(2, 5)  # 100 to 10000
f_decade = []
for d in decades:
    f_decade.extend(np.logspace(d, d+1, 11)[:-1])
f_decade.append(10000)
f_decade = np.array(f_decade)

axes[0, 1].stem(f_decade, np.ones_like(f_decade), basefmt=' ')
axes[0, 1].set_xscale('log')
axes[0, 1].set_xlabel('Frequency (Hz)')
axes[0, 1].set_title('Decade Sweep: 10 points per decade')

# Octave sweep
octaves = np.arange(0, 8)  # 8 octaves from 100Hz
f_octave = 100 * 2**octaves
axes[1, 0].stem(f_octave, np.ones_like(f_octave), basefmt=' ')
axes[1, 0].set_xscale('log')
axes[1, 0].set_xlabel('Frequency (Hz)')
axes[1, 0].set_title('Octave Sweep: 1 point per octave')

# List sweep
f_list = np.array([50, 60, 100, 120, 1000, 10000, 100000])
axes[1, 1].stem(f_list, np.ones_like(f_list), basefmt=' ')
axes[1, 1].set_xscale('log')
axes[1, 1].set_xlabel('Frequency (Hz)')
axes[1, 1].set_title('List Sweep: Custom frequencies')

plt.tight_layout()
plt.show()

## 3. RC Low-Pass Filter Analysis

First-order RC low-pass filter:

$$H(j\omega) = \frac{1}{1 + j\omega RC} = \frac{1}{1 + j\frac{f}{f_c}}$$

Where cutoff frequency $f_c = \frac{1}{2\pi RC}$

- Magnitude: $|H| = \frac{1}{\sqrt{1 + (f/f_c)^2}}$
- Phase: $\angle H = -\arctan(f/f_c)$

In [ ]:
# RC Low-Pass Filter AC Analysis
R = 1000    # 1kΩ
C = 100e-9  # 100nF
fc = 1 / (2 * np.pi * R * C)  # Cutoff frequency

print(f"RC Low-Pass Filter:")
print(f"  R = {R} Ω")
print(f"  C = {C*1e9:.0f} nF")
print(f"  Cutoff frequency (fc) = {fc:.1f} Hz")

In [ ]:
# Create circuit and run AC analysis
netlist = f'''
{{
  "name": "RC Low-Pass Filter",
  "components": [
    {{"type": "V", "name": "Vin", "nodes": ["in", "0"], "value": 1.0, "ac": 1.0}},
    {{"type": "R", "name": "R1", "nodes": ["in", "out"], "value": {R}}},
    {{"type": "C", "name": "C1", "nodes": ["out", "0"], "value": {C}}}
  ]
}}
'''

circuit = pulsim.parse_netlist_string(netlist)

# Configure AC options
ac_options = pulsim.ACOptions()
ac_options.sweep_type = pulsim.FrequencySweepType.Decade
ac_options.fstart = 10       # 10 Hz
ac_options.fstop = 100000    # 100 kHz
ac_options.npoints = 20      # 20 points per decade

# Run AC analysis
ac_result = pulsim.ac_analysis(circuit, ac_options)

print(f"AC Analysis completed:")
print(f"  Frequency points: {ac_result.num_frequencies()}")
print(f"  Signals: {ac_result.signal_names}")

In [ ]:
# Extract and plot results
frequencies = np.array(ac_result.frequencies)

# Find output voltage signal index
out_idx = ac_result.signal_names.index('V(out)')

# Get magnitude and phase
magnitude_db = np.array([ac_result.magnitude_db(i, out_idx) for i in range(len(frequencies))])
phase_deg = np.array([ac_result.phase_deg(i, out_idx) for i in range(len(frequencies))])

# Theoretical values
H_theory = 1 / (1 + 1j * frequencies / fc)
mag_theory_db = 20 * np.log10(np.abs(H_theory))
phase_theory_deg = np.angle(H_theory, deg=True)

# Bode plot
fig, axes = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

# Magnitude
axes[0].semilogx(frequencies, magnitude_db, 'b-', linewidth=2, label='Simulated')
axes[0].semilogx(frequencies, mag_theory_db, 'r--', linewidth=2, label='Theoretical')
axes[0].axvline(x=fc, color='g', linestyle=':', label=f'fc = {fc:.0f} Hz')
axes[0].axhline(y=-3, color='gray', linestyle=':', alpha=0.5)
axes[0].set_ylabel('Magnitude (dB)')
axes[0].set_title('RC Low-Pass Filter Bode Plot')
axes[0].legend()
axes[0].grid(True, which='both', alpha=0.5)
axes[0].set_ylim([-60, 5])

# Phase
axes[1].semilogx(frequencies, phase_deg, 'b-', linewidth=2, label='Simulated')
axes[1].semilogx(frequencies, phase_theory_deg, 'r--', linewidth=2, label='Theoretical')
axes[1].axvline(x=fc, color='g', linestyle=':', label=f'fc = {fc:.0f} Hz')
axes[1].axhline(y=-45, color='gray', linestyle=':', alpha=0.5)
axes[1].set_xlabel('Frequency (Hz)')
axes[1].set_ylabel('Phase (degrees)')
axes[1].legend()
axes[1].grid(True, which='both', alpha=0.5)
axes[1].set_ylim([-100, 10])

plt.tight_layout()
plt.show()

print(f"\nAt cutoff frequency ({fc:.0f} Hz):")
print(f"  Magnitude: -3 dB (70.7% of input)")
print(f"  Phase: -45°")

## 4. Bode Plot Generation

Pulsim provides helper functions for Bode plot data extraction:

```python
BodeData:
    frequencies      # Hz
    magnitude_db     # dB
    phase_deg        # degrees
    
    # Stability margins (if applicable)
    gain_margin_db
    phase_margin_deg
    gain_crossover_freq
    phase_crossover_freq
```

In [ ]:
# Extract Bode data from AC result
# Transfer function: V(out) / V(in)
in_idx = ac_result.signal_names.index('V(in)')
out_idx = ac_result.signal_names.index('V(out)')

# Get transfer function magnitude and phase
tf_mag_db = np.array([ac_result.transfer_magnitude_db(i, out_idx, in_idx) 
                      for i in range(len(frequencies))])
tf_phase_deg = np.array([ac_result.transfer_phase_deg(i, out_idx, in_idx) 
                         for i in range(len(frequencies))])

# Create Bode data using extract_bode_data
bode = pulsim.extract_bode_data(ac_result, 'V(out)', 'V(in)')

print("Bode Data Extracted:")
print(f"  Frequency range: {bode.frequencies[0]:.0f} Hz to {bode.frequencies[-1]:.0f} Hz")
print(f"  DC gain: {bode.magnitude_db[0]:.2f} dB")
print(f"  High frequency roll-off: {bode.magnitude_db[-1]:.1f} dB")

In [ ]:
# Helper function for nice Bode plots
def plot_bode(frequencies, magnitude_db, phase_deg, title="Bode Plot", 
              fc=None, margins=None):
    """Create a standard Bode plot."""
    fig, axes = plt.subplots(2, 1, figsize=(12, 8), sharex=True)
    
    # Magnitude
    axes[0].semilogx(frequencies, magnitude_db, 'b-', linewidth=2)
    axes[0].axhline(y=0, color='k', linestyle='-', linewidth=0.5)
    if fc:
        axes[0].axvline(x=fc, color='r', linestyle='--', alpha=0.7, label=f'fc = {fc:.0f} Hz')
        axes[0].axhline(y=-3, color='gray', linestyle=':', alpha=0.5, label='-3dB')
    axes[0].set_ylabel('Magnitude (dB)')
    axes[0].set_title(title)
    axes[0].grid(True, which='both', alpha=0.5)
    if fc:
        axes[0].legend()
    
    # Phase
    axes[1].semilogx(frequencies, phase_deg, 'b-', linewidth=2)
    axes[1].axhline(y=0, color='k', linestyle='-', linewidth=0.5)
    axes[1].axhline(y=-180, color='r', linestyle='--', alpha=0.5)
    if fc:
        axes[1].axvline(x=fc, color='r', linestyle='--', alpha=0.7)
    axes[1].set_xlabel('Frequency (Hz)')
    axes[1].set_ylabel('Phase (degrees)')
    axes[1].grid(True, which='both', alpha=0.5)
    
    # Add stability margins if provided
    if margins:
        if margins.get('phase_margin'):
            pm = margins['phase_margin']
            fc_pm = margins['gain_crossover']
            axes[1].annotate(f'PM = {pm:.1f}°', xy=(fc_pm, -180 + pm),
                           xytext=(fc_pm*2, -180 + pm + 20),
                           arrowprops=dict(arrowstyle='->', color='green'),
                           fontsize=10, color='green')
    
    plt.tight_layout()
    return fig, axes

# Use the helper
plot_bode(bode.frequencies, bode.magnitude_db, bode.phase_deg,
          title='RC Low-Pass Filter Transfer Function', fc=fc)
plt.show()

## 5. Stability Margins

For feedback systems, stability margins indicate robustness:

- **Phase Margin (PM)**: Phase above -180° at 0dB crossover
- **Gain Margin (GM)**: Gain below 0dB at -180° crossover

$$\text{PM} = 180° + \angle H(j\omega_{gc})$$
$$\text{GM} = -|H(j\omega_{pc})|_{dB}$$

In [ ]:
# Create a control loop transfer function
# Simple proportional controller with plant: G(s) = K / (s(s+1)(s+10))

# Simulate using equivalent circuit
# H(s) = 100 / (s³ + 11s² + 10s)
# Using gyrators and integrators for complex transfer function

# For demonstration, let's analyze a 2nd order system
# H(s) = wn² / (s² + 2ζwn·s + wn²)

wn = 1000 * 2 * np.pi  # Natural frequency: 1kHz
zeta = 0.3  # Underdamped

# RLC circuit implements this!
# wn = 1/sqrt(LC), zeta = R/(2)*sqrt(C/L)
L = 10e-3     # 10mH
C = 1 / (wn**2 * L)  # For wn = 1kHz
R = 2 * zeta * np.sqrt(L / C)

print(f"2nd Order System (RLC):")
print(f"  Natural frequency: {wn/(2*np.pi):.0f} Hz")
print(f"  Damping ratio: {zeta}")
print(f"  R = {R:.2f} Ω, L = {L*1e3:.1f} mH, C = {C*1e6:.2f} µF")

In [ ]:
# 2nd order RLC filter
netlist = f'''
{{
  "name": "RLC 2nd Order Filter",
  "components": [
    {{"type": "V", "name": "Vin", "nodes": ["in", "0"], "value": 1.0, "ac": 1.0}},
    {{"type": "R", "name": "R1", "nodes": ["in", "n1"], "value": {R}}},
    {{"type": "L", "name": "L1", "nodes": ["n1", "out"], "value": {L}}},
    {{"type": "C", "name": "C1", "nodes": ["out", "0"], "value": {C}}}
  ]
}}
'''

circuit = pulsim.parse_netlist_string(netlist)

ac_options = pulsim.ACOptions()
ac_options.sweep_type = pulsim.FrequencySweepType.Decade
ac_options.fstart = 10
ac_options.fstop = 100000
ac_options.npoints = 30

ac_result = pulsim.ac_analysis(circuit, ac_options)

# Extract Bode data
bode = pulsim.extract_bode_data(ac_result, 'V(out)', 'V(in)')

# Calculate stability margins
pulsim.calculate_stability_margins(bode)

# Theoretical transfer function
f = np.array(bode.frequencies)
s = 1j * 2 * np.pi * f
H_theory = wn**2 / (s**2 + 2*zeta*wn*s + wn**2)

fig, axes = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

# Magnitude
axes[0].semilogx(f, bode.magnitude_db, 'b-', linewidth=2, label='Simulated')
axes[0].semilogx(f, 20*np.log10(np.abs(H_theory)), 'r--', linewidth=2, label='Theoretical')
axes[0].axvline(x=wn/(2*np.pi), color='g', linestyle=':', label=f'fn = {wn/(2*np.pi):.0f} Hz')
axes[0].set_ylabel('Magnitude (dB)')
axes[0].set_title(f'RLC Filter: ζ = {zeta} (Underdamped - resonant peak)')
axes[0].legend()
axes[0].grid(True, which='both', alpha=0.5)

# Phase
axes[1].semilogx(f, bode.phase_deg, 'b-', linewidth=2, label='Simulated')
axes[1].semilogx(f, np.angle(H_theory, deg=True), 'r--', linewidth=2, label='Theoretical')
axes[1].axvline(x=wn/(2*np.pi), color='g', linestyle=':')
axes[1].axhline(y=-90, color='gray', linestyle=':', alpha=0.5, label='-90° at fn')
axes[1].set_xlabel('Frequency (Hz)')
axes[1].set_ylabel('Phase (degrees)')
axes[1].legend()
axes[1].grid(True, which='both', alpha=0.5)

plt.tight_layout()
plt.show()

# Calculate resonant peak
peak_idx = np.argmax(bode.magnitude_db)
peak_freq = bode.frequencies[peak_idx]
peak_gain = bode.magnitude_db[peak_idx]

print(f"\n2nd Order System Analysis:")
print(f"  Resonant frequency: {peak_freq:.0f} Hz")
print(f"  Peak gain: {peak_gain:.1f} dB")
print(f"  Q factor: {1/(2*zeta):.1f}")

## 6. LC and RLC Filters

Common filter types for power electronics:

| Type | Transfer Function | Use |
|------|-------------------|-----|
| LC Low-Pass | $\frac{1}{LCs^2 + 1}$ | Output filter (undamped) |
| RLC Low-Pass | $\frac{1}{LCs^2 + RCs + 1}$ | Output filter (damped) |
| LC Notch | $\frac{LCs^2 + 1}{LCs^2 + RCs + 1}$ | EMI filter |

In [ ]:
# Compare filter damping ratios
L = 100e-6   # 100µH
C = 100e-6   # 100µF
f0 = 1 / (2 * np.pi * np.sqrt(L * C))  # Resonant frequency

print(f"LC Filter:")
print(f"  L = {L*1e6:.0f} µH")
print(f"  C = {C*1e6:.0f} µF")
print(f"  Resonant frequency: {f0:.0f} Hz")

# Different damping resistors
R_critical = 2 * np.sqrt(L / C)  # Critical damping
print(f"  Critical damping resistance: {R_critical:.2f} Ω")

In [ ]:
# Compare different damping levels
damping_configs = [
    (0.001, 'Undamped (R=1mΩ)'),
    (R_critical * 0.3, f'Underdamped (ζ=0.3)'),
    (R_critical, f'Critically damped (ζ=1.0)'),
    (R_critical * 2, f'Overdamped (ζ=2.0)'),
]

fig, axes = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

for R, label in damping_configs:
    netlist = f'''
    {{
      "name": "LC Filter R={R}",
      "components": [
        {{"type": "V", "name": "Vin", "nodes": ["in", "0"], "value": 1.0, "ac": 1.0}},
        {{"type": "R", "name": "R1", "nodes": ["in", "n1"], "value": {R}}},
        {{"type": "L", "name": "L1", "nodes": ["n1", "out"], "value": {L}}},
        {{"type": "C", "name": "C1", "nodes": ["out", "0"], "value": {C}}}
      ]
    }}
    '''
    
    circuit = pulsim.parse_netlist_string(netlist)
    
    ac_options = pulsim.ACOptions()
    ac_options.sweep_type = pulsim.FrequencySweepType.Decade
    ac_options.fstart = 10
    ac_options.fstop = 100000
    ac_options.npoints = 30
    
    ac_result = pulsim.ac_analysis(circuit, ac_options)
    bode = pulsim.extract_bode_data(ac_result, 'V(out)', 'V(in)')
    
    axes[0].semilogx(bode.frequencies, bode.magnitude_db, linewidth=2, label=label)
    axes[1].semilogx(bode.frequencies, bode.phase_deg, linewidth=2, label=label)

axes[0].axvline(x=f0, color='gray', linestyle=':', alpha=0.5)
axes[0].set_ylabel('Magnitude (dB)')
axes[0].set_title(f'LC Filter with Different Damping (f0 = {f0:.0f} Hz)')
axes[0].legend()
axes[0].grid(True, which='both', alpha=0.5)
axes[0].set_ylim([-80, 30])

axes[1].axvline(x=f0, color='gray', linestyle=':', alpha=0.5)
axes[1].set_xlabel('Frequency (Hz)')
axes[1].set_ylabel('Phase (degrees)')
axes[1].legend()
axes[1].grid(True, which='both', alpha=0.5)

plt.tight_layout()
plt.show()

## 7. Control Loop Analysis

AC analysis is essential for power converter control design:

1. **Plant transfer function**: Output/duty cycle
2. **Loop gain**: Controller × Plant × Feedback
3. **Stability margins**: PM > 45°, GM > 6dB typical

In [ ]:
# Buck converter output filter frequency response
# Plant: Gvd(s) = Vin / (LCs² + s/R + 1)

Vin = 12
L = 100e-6
C = 100e-6
R_load = 5

f0 = 1 / (2 * np.pi * np.sqrt(L * C))
Q = R_load * np.sqrt(C / L)

print(f"Buck Converter Output Filter:")
print(f"  Vin = {Vin} V")
print(f"  L = {L*1e6:.0f} µH")
print(f"  C = {C*1e6:.0f} µF")
print(f"  R_load = {R_load} Ω")
print(f"  \nResonant frequency: {f0:.0f} Hz")
print(f"  Quality factor Q: {Q:.1f}")
print(f"  Peak at resonance: {20*np.log10(Q):.1f} dB")

In [ ]:
# Simulate buck converter small-signal response
netlist = f'''
{{
  "name": "Buck Converter Small-Signal Model",
  "components": [
    {{"type": "V", "name": "Vd", "nodes": ["d", "0"], "value": {Vin}, "ac": 1.0}},
    {{"type": "L", "name": "L1", "nodes": ["d", "out"], "value": {L}}},
    {{"type": "C", "name": "C1", "nodes": ["out", "0"], "value": {C}}},
    {{"type": "R", "name": "Rload", "nodes": ["out", "0"], "value": {R_load}}}
  ]
}}
'''

circuit = pulsim.parse_netlist_string(netlist)

ac_options = pulsim.ACOptions()
ac_options.sweep_type = pulsim.FrequencySweepType.Decade
ac_options.fstart = 10
ac_options.fstop = 1000000
ac_options.npoints = 30

ac_result = pulsim.ac_analysis(circuit, ac_options)
bode = pulsim.extract_bode_data(ac_result, 'V(out)', 'V(d)')

# Plot control-to-output transfer function
fig, axes = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

axes[0].semilogx(bode.frequencies, bode.magnitude_db, 'b-', linewidth=2)
axes[0].axvline(x=f0, color='r', linestyle='--', label=f'f0 = {f0:.0f} Hz')
axes[0].axhline(y=20*np.log10(Vin), color='g', linestyle=':', label=f'DC gain = {20*np.log10(Vin):.1f} dB')
axes[0].set_ylabel('Magnitude (dB)')
axes[0].set_title('Buck Converter: Control-to-Output Transfer Function Gvd(s)')
axes[0].legend()
axes[0].grid(True, which='both', alpha=0.5)

axes[1].semilogx(bode.frequencies, bode.phase_deg, 'b-', linewidth=2)
axes[1].axvline(x=f0, color='r', linestyle='--')
axes[1].axhline(y=-180, color='gray', linestyle=':', alpha=0.5)
axes[1].set_xlabel('Frequency (Hz)')
axes[1].set_ylabel('Phase (degrees)')
axes[1].grid(True, which='both', alpha=0.5)

plt.tight_layout()
plt.show()

print(f"\nControl Design Implications:")
print(f"  - Phase drops to -180° at resonance")
print(f"  - Crossover frequency should be < f0/5 = {f0/5:.0f} Hz for stability")
print(f"  - Or use type-III compensator to add phase boost")

In [ ]:
# Design considerations summary
print("Control Loop Design Guidelines:")
print("="*50)
print("")
print("Phase Margin (PM):")
print("  - PM > 45° for good transient response")
print("  - PM > 60° for minimal overshoot")
print("  - PM < 30° indicates potential instability")
print("")
print("Gain Margin (GM):")
print("  - GM > 6 dB recommended")
print("  - GM > 10 dB for robust design")
print("")
print("Crossover Frequency (fc):")
print("  - fc < fsw/10 to avoid switching noise")
print("  - fc > 5× load transient for good regulation")
print("  - fc typically 1/10 to 1/5 of switching frequency")

## Summary

### AC Analysis in Pulsim

```python
# Basic workflow
circuit = pulsim.parse_netlist_string(netlist)
ac_options = pulsim.ACOptions()
ac_options.sweep_type = pulsim.FrequencySweepType.Decade
ac_options.fstart = 10
ac_options.fstop = 1e6
ac_options.npoints = 20

ac_result = pulsim.ac_analysis(circuit, ac_options)
bode = pulsim.extract_bode_data(ac_result, 'V(out)', 'V(in)')
```

### Key Functions

| Function | Purpose |
|----------|--------|
| `ac_analysis()` | Run AC frequency sweep |
| `ACAnalyzer` | Advanced AC analysis with operating point |
| `extract_bode_data()` | Get magnitude/phase arrays |
| `calculate_stability_margins()` | Compute PM and GM |

### Filter Design Quick Reference

| Order | Type | Roll-off | Phase at fc |
|-------|------|----------|-------------|
| 1st | RC | -20 dB/dec | -45° |
| 2nd | LC | -40 dB/dec | -90° (ζ dependent) |
| 2nd | RLC | -40 dB/dec | -90° to -180° |

**Next:** [Transformers and Magnetics](09_transformers.ipynb)